# Optimizing Models for Serving

## Build the Graph Optimization Tool

Note, this is for reference.  The full build takes ~ 30 minutes.  Your image already contains prebuilt binaries for graph transform tools.

In [ ]:
%%bash
## install bazel
#sudo apt-get install -y pkg-config zip g++ zlib1g-dev unzip python
#wget https://github.com/bazelbuild/bazel/releases/download/0.16.0/bazel-0.16.0-installer-linux-x86_64.sh
#chmod +x bazel-0.16.0-installer-linux-x86_64.sh
#./bazel-0.16.0-installer-linux-x86_64.sh --user

## build gtt and summarize graph
#sudo apt-get install -y python3-numpy python3-dev python3-pip python3-wheel
#git clone https://github.com/tensorflow/tensorflow
#cd tensorflow
#git checkout r1.10
#bazel build tensorflow/tools/graph_transforms:transform_graph
#bazel build tensorflow/tools/graph_transforms:summarize_graph

## Download the SavedModel

In [ ]:
%%bash
gsutil cp -r gs://tfs-data/models/mnist_cnn_model .

## Inspect the model

In [15]:
%%bash
tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph --in_graph=mnist_cnn_model/graph.pbtxt

Found 3 possible inputs: (name=enqueue_input/Placeholder, type=int64(9), shape=<unknown>) (name=enqueue_input/Placeholder_1, type=float(1), shape=<unknown>) (name=enqueue_input/Placeholder_2, type=int32(3), shape=<unknown>) 
Found 11 variables: (name=global_step, type=int64(9), shape=[]) (name=conv1/W, type=float(1), shape=[5,5,1,64]) (name=conv1/b, type=float(1), shape=[64]) (name=conv2/W, type=float(1), shape=[5,5,64,64]) (name=conv2/b, type=float(1), shape=[64]) (name=Dense/W, type=float(1), shape=[3136,1024]) (name=Dense/b, type=float(1), shape=[1024]) (name=Output/W, type=float(1), shape=[1024,10]) (name=Output/b, type=float(1), shape=[10]) (name=accuracy/total, type=float(1), shape=[]) (name=accuracy/count, type=float(1), shape=[]) 
Found 21 possible outputs: (name=global_step/cond/switch_t, op=Identity) (name=global_step/cond/switch_f, op=Identity) (name=global_step/add, op=Add) (name=enqueue_input/random_shuffle_queue_EnqueueMany, op=QueueEnqueueManyV2) (name=enqueue_input/rand

2018-08-06 22:54:16.364390: W tensorflow/tools/graph_transforms/summarize_graph_main.cc:225] Decoding Tensor failed for nodeglobal_step
2018-08-06 22:54:16.364485: W tensorflow/tools/graph_transforms/summarize_graph_main.cc:225] Decoding Tensor failed for nodeconv1/W
2018-08-06 22:54:16.364507: W tensorflow/tools/graph_transforms/summarize_graph_main.cc:225] Decoding Tensor failed for nodeconv1/b
2018-08-06 22:54:16.364524: W tensorflow/tools/graph_transforms/summarize_graph_main.cc:225] Decoding Tensor failed for nodeconv2/W
2018-08-06 22:54:16.364533: W tensorflow/tools/graph_transforms/summarize_graph_main.cc:225] Decoding Tensor failed for nodeconv2/b
2018-08-06 22:54:16.364549: W tensorflow/tools/graph_transforms/summarize_graph_main.cc:225] Decoding Tensor failed for nodeDense/W
2018-08-06 22:54:16.364558: W tensorflow/tools/graph_transforms/summarize_graph_main.cc:225] Decoding Tensor failed for nodeDense/b
2018-08-06 22:54:16.364582: W tensorflow/tools/graph_transforms/summariz

## Run the Graph Optimization Tool on the model

In [23]:
%%bash

tensorflow/bazel-bin/tensorflow/tools/graph_transforms/transform_graph \
--in_graph=mnist_cnn_model/graph.pbtxt \
--out_graph=mnist_cnn_model/optimized_model.pb \
--inputs='enqueue_input/Placeholder' \
--outputs='softmax' \
--transforms='
strip_unused_nodes(type=float, shape="1,28,28,1")
remove_nodes(op=Identity, op=CheckNumerics)
'

2018-08-06 23:29:23.986839: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying strip_unused_nodes
2018-08-06 23:29:23.988389: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes


## Inspect the model again

In [ ]:
%%bash
tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph --in_graph=mnist_cnn_model/graph.pbtxt

## Freeze the graph

In [25]:
%%bash
tensorflow/bazel-bin/tensorflow/python/tools/freeze_graph \
--input_graph=mnist_cnn_model/optimized_model.pb \
--input_checkpoint=mnist_cnn_model/model.ckpt-20000 \
--output_graph=mnist_cnn_model/frozen_graph.pb \
--output_node_names=softmax

/home/lramsey/miniconda3/envs/tfs/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lramsey/miniconda3/envs/tfs/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Traceback (most recent call last):
  File "/home/lramsey/serving-opt/tensorflow/bazel-bin/tensorflow/python/tools/freeze_graph.runfiles/org_tensorflow/tensorflow/python/tools/freeze_graph.py", line 382, in <module>
    run_main()
  File "/home/lramsey/serving-opt/tensorflow/bazel-bin/tensorflow/python/tools/freeze_graph.runfiles/org_tensorflow/tensorflow/python/tools/freeze_graph.py", line 379, in run_main
    app.run(main=my_main, argv=[sys.argv[0]] + unparsed)
  File "/home/lramsey/serving-opt/tensorflow/bazel-bin/tensorflow/python/tools/freeze_graph.runfiles/org_tensorflow/tensorflow/py

In [21]:
%%bash
tensorflow/bazel-bin/tensorflow/tools/graph_transforms/summarize_graph --in_graph=mnist_cnn_model/frozen_graph.pb

No inputs spotted.
No variables spotted.
Found 1 possible outputs: (name=softmax, op=Softmax) 
Found 3326680 (3.33M) const parameters, 0 (0) variable parameters, and 0 control_edges
Op types used: 16 Const, 8 Identity, 6 Add, 3 Relu, 3 Reshape, 2 Conv2D, 2 MatMul, 2 MaxPool, 2 Mul, 1 Floor, 1 QueueDequeueManyV2, 1 RandomShuffleQueueV2, 1 RandomUniform, 1 RealDiv, 1 Softmax, 1 Sub
To use with tensorflow/tools/benchmark:benchmark_model try these arguments:
bazel run tensorflow/tools/benchmark:benchmark_model -- --graph=mnist_cnn_model/frozen_graph.pb --show_flops --input_layer= --input_layer_type= --input_layer_shape= --output_layer=softmax
